Trabajo práctico II

## Detección de objetos con Deep Learning, CNN y YOLO

Braian D'Aleo - Lic. en Ciencia de Datos - UGR 2024

Para este trabajo, preferi usar el entorno local para acelerar los procesos. En colab procesar un video de 20 segundos me demora unos 5 minutos, usando mi entorno local puedo hacer lo mismo en 1 minuto.

In [ ]:
# basics
import pandas as pd
import numpy as np

%matplotlib inline
from IPython.display import Video, display, HTML, Image
import cv2
import matplotlib.pyplot as plt
import torch

# model
# !pip install lapx
# !pip install ultralytics
from ultralytics import YOLO
model = YOLO("yolov8n.pt")

# Active tracking
import os
import imageio
from collections import defaultdict, Counter

Primero descargamos los archivos necesarios, para esto usamos una funcion de descarga desde Google Drive

# Etapa 1 - Detección de objetos

Veamos la imagen original con la que vamos a trabajar:

In [ ]:
image_path = 'source/photo.png'
Image(image_path)

Comenzamos con la primer deteccion de imagen, aprovechando el poder de computo local vamos a usar el mayor muestreo disponible para el modelo: 1080

In [ ]:
image_save_path = 'photo-detection-1.png'

results = model(image_path, imgsz=1920) # Donde transcurre todo
for result in results:
    result.save(image_save_path)

Image(image_save_path)

Veamos una comparacion ingresando al modelo diferentes parametros:

In [ ]:
Image('source/ejemplo.png')

Podemos observar que a primeras el modelo pre-entrenado funciona correctamente con sus parametros por defecto, aunque si tenemos disponibilidad computacional y podemos avanzar un poco vamos a notar diferencias. Notese la cantidad de objetos que se detectaron de mas en la segunda imagen. Esto se debe a que el modelo por defecto busca un punto de equilibrio entre rendimiento y resultados. 

Vamos con el procesamiento de video, primero veamos el video con el que vamos a trabajar:

In [ ]:
video_path = 'source/video.mp4'
Video(video_path, width=1000, embed=True)

Se trata de un video un tanto complicado, hay demasiados objetos y muy diversos, la camara esta en constante movimiento como asi tambien la clase mayoritaria (personas), esto impone un desafio para el modelo, vamos a ver como responde.

Sobre la detección de objetos en video:

Para la deteccion de videos vamos a utilizar **CV2** para leer el video, y **imageio** para grabar los frames con las detecciones. Esto es asi, ya que de forma nativa CV2 no tiene por defecto soporte para el codec x264. Aca tambien voy a comentar en detalle el codigo, para no repetirlo constantemente:

In [ ]:
input_path = video_path     # Ruta de entrada
output_path = 'video-detection-1.mp4'      # Ruta de salida

if os.path.exists(output_path):
    os.remove(output_path)      # Si el archivo de salida existe, que lo borre

cap = cv2.VideoCapture(input_path)      # Abrimos el video
fps = cap.get(cv2.CAP_PROP_FPS)     # Obtenemos cuadros por segundo
out = imageio.get_writer(output_path, fps=fps, codec='libx264')     # Abrimos el grabador de video

while cap.isOpened():
    status, frame = cap.read()      # Mientras existan cuadros
    if not status:
        break

    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)      # Pasamos al modelo el frame en RGB
    results = model(frame, imgsz=1920)      # Aumentamos la resolucion al modelo
    annotated_frame = results[0].plot()     # Incorpora al frame los resultados
    out.append_data(annotated_frame)        # Grabo el frame

cap.release()       # Libero recursos de entrada
out.close()     # Libero recursos de salida

Veamos el video resultante:

In [ ]:
Video(output_path, width=1000, embed=True)

Nuevamente, podemos apreciar que el modelo reconoce objetos bastante bien. Con estos pequeños ajustes podemos lograr detectar objetos pequeños y de dificil reconocimiento, un telefono celular, una pequeña mochila. Por otro lado vemos que el uso de CPU es absoluto, lo que asegura el uso del 100% de nuestros recursos:

In [ ]:
Image('source/core.png')

# Etapa 2 - Personalización de etiquetas

Para este paso vamos a personalizar algunas etiquetas y colores, guardar estos valores y persistirlos durante el estadio de trabajo del modelo.

In [ ]:
conf_threshold = 0.6  # Valor umbral principal

colors = {
    'Automovil': (255, 0, 0),  # Rojo
    'Persona': (0, 255, 0),    # Verde
    'Camion': (0, 0, 255),     # Azul
    'Colectivo': (255, 255, 0), # Amarillo
    'Cartera': (0, 255, 255),   # Cyan
    'Semaforo': (255, 0, 255),  # Magenta
    'Moto': (0, 255, 255),       # Cyan
    'Mochila': (255, 255, 255),   # Blanco
}

labels = {
    'car': 'Automovil',
    'person': 'Persona',
    'truck': 'Camion',
    'bus': 'Colectivo',
    'handbag': 'Cartera',
    'traffic light': 'Semaforo',
    'motorcycle': 'Moto',
    'backpack': 'Mochila',
    'bench': 'Banco'
}

default_color = (0, 0, 0)  # Color por defecto para etiquetas no especificadas
default_label = 'Desconocido'    # Etiqueta por defecto para categorías no especificadas

Las clases y su significado:



```
0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell phone', 68: 'microwave', 69: 'oven', 70: 'toaster', 71: 'sink', 72: 'refrigerator', 73: 'book', 74: 'clock', 75: 'vase', 76: 'scissors', 77: 'teddy bear', 78: 'hair drier', 79: 'toothbrush'
```



In [ ]:
def detect_image(img):
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # Me aseguro que sea RGB
    results = model(img_rgb, imgsz=1920, conf=0.6) # Confianza > 60

    for result in results:
        for box in result.boxes:
            conf = box.conf.item()
            cls = box.cls.item() # Extraigo las clases
            x1, y1, x2, y2 = map(int, box.xyxy[0]) # Mapeo los recuadros
            yolo_label = model.names[int(cls)]
            label = labels.get(yolo_label, default_label) # Utilizo mis valores
            color = colors.get(label, default_color) # Mis colores
            confidence = conf * 100
            text = f'{label} {confidence:.1f}%' # Informaciôn de etiqueta

            # Cuadro y etiqueta en la imagen
            cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
            cv2.putText(img, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
    return img

image_save_path = 'photo-detection-2.png'
img_original = cv2.imread(image_path) # Me traigo la imagen
img = detect_image(img_original) # Paso la imagen y la confianza

cv2.imwrite(image_save_path, img)

Image(image_save_path)

Logramos los objetivos pedidos, etiquetar con nuevas etiquetas, colores y porcentaje de confianza. La observacion aqui, es que pasando este umbral, el modelo va a lo seguro, se refina. Elimina los objetos que considera ruido y se enfoca en los mejor explicados. Vamos con la etapa de video, y para tal fin vamos a reutilizar la funcion que utilizamos en la imagen:

In [ ]:
input_path = video_path
output_path = 'video-detection-2.mp4'

if os.path.exists(output_path):
    os.remove(output_path)

cap = cv2.VideoCapture(input_path)
fps = cap.get(cv2.CAP_PROP_FPS)
out = imageio.get_writer(output_path, fps=fps, codec='libx264')

while cap.isOpened():
    status, frame = cap.read()
    if not status:
        break

    frame = detect_image(frame)  # Reutilizo la funcion de imagen
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    out.append_data(frame)  # Grabo

cap.release()
out.close()

Veamos el video resultante:

In [ ]:
Video(output_path, width=1000, embed=True)

Nuevamente observamos los cambios en las etiquetas, los colores y los objetos detectados por ensima del threshold asignado.

# Etapa 3 - Segmentación y detección de objetos

Para esta etapa se nos pidio contabilizar los objetos unicos en imagen y en video, para la imagen fue simple, ya que solo habia que obtener el mayor y el menor:

In [ ]:
def process_image(img_path, labels, default_label='Other'):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = model(img, imgsz=1920)  # Asegúrate de que el modelo está cargado correctamente

    # Contar la cantidad de objetos detectados de cada clase
    object_counts = Counter()
    for result in results:
        for box in result.boxes:
            cls = box.cls.item()
            yolo_label = model.names[int(cls)]
            # Aplicar etiquetas personalizadas inmediatamente
            custom_label = labels.get(yolo_label, default_label)
            object_counts[custom_label] += 1

    if not object_counts:
        return

    # Identificar las clases con mayor y menor cantidad de objetos
    most_common_class, least_common_class = object_counts.most_common(1)[0][0], object_counts.most_common()[-1][0]

    # Procesar y resaltar solo los objetos de las clases identificadas
    for result in results:
        for box in result.boxes:
            cls = box.cls.item()
            yolo_label = model.names[int(cls)]
            custom_label = labels.get(yolo_label, default_label)

            if custom_label == most_common_class:
                color = (255, 0, 0)  # Azul
            elif custom_label == least_common_class:
                color = (0, 0, 255)  # Rojo
            else:
                continue  # Saltar los objetos que no son de las clases identificadas

            conf = box.conf.item()
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            confidence = conf * 100
            text = f'{custom_label} {confidence:.1f}%'

            # Dibujar cuadro de delimitación y texto
            cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
            cv2.putText(img, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

    return img

image_save_path = 'photo-detection-3.png'
img = process_image(image_path, labels)
img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
cv2.imwrite(image_save_path, img)

Image(image_save_path)

Los resultados obtenidos con los ajustes propuestos fueron muy buenos, siendo capaz el modelo de detectar como objeto menos frecuente la mochila que apenas se ve en la imagen. Esto, nuevamente se debe a la ampliacion de la muestra pasada al modelo.

Para la parte del video, que incluye object tracking, mi primer acercamiento fue a traves de SORT, para ello tuve que modificar un repo de GIT, y hacer mil ajustes innecesarios. Por suerte leyendo la documentacion de Yolo vi que Yolov8 tenia disponible el traker de forma nativa, esta implementacion fue extremadamente mas simple que el proceso anterior, y muchos menos costosa computacionalmente.

Para un video de solo unos segundos no se justifica, podrimos revisar manualmente las figuras detectadas, aunque pordriamos luego reutilizar la funcion para cualquier tipo de video y duración.

Esta funcion basicamente abre el video cuadro a cuadro y genera todos los resultados en tracked_objects. Estos tienen la posicion, el id, la clase y todo lo que se le pida que guarde al modelo. Por otro lado se genera un diccionario de clases unicas, este va evaluando por id las clases y las cuenta. Posteriormente con max y min obtenemos la clase (int) que mas y menos frecuencia tiene. Vale aclarar que una persona que va del punto A al punto B se cuenta como unica persona. 

Veamos como funciona el seguimiento de YoloV8 por defecto:

In [ ]:
# input_path = video_path
input_path = 'source/video.mp4'
output_path = "video-detection-3.mp4"

model = YOLO("yolov8n.pt")

if os.path.exists(output_path):
    os.remove(output_path)

cap = cv2.VideoCapture(input_path)
fps = cap.get(cv2.CAP_PROP_FPS)
out = imageio.get_writer(output_path, fps=fps, codec='libx264')
total_fr = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
fr = 0
while cap.isOpened():
    status, frame = cap.read()

    if status:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # Enviamos RGB para evitar salida BGR
        results = model.track(source=frame, persist=True, imgsz=1920) # Persistiendo tracks sobre frames
        frame = results[0].plot()
        fr += 1
        print(f'Frame {fr} / {total_fr}')
        out.append_data(frame)
    else:
        break

cap.release()
out.close()

Veamos los resultados obtenidos:

In [ ]:
Video(output_path, width=1000, embed=True)

Los resultados a primeras son muy buenos, manteniendo los ids en la mayoria de casos. Vemos que no resiste el solapamiento de personas con demasiada eficacia, aunque en terminos generales es muy aceptable.

Con respecto al contador de clases unicas, decidi separar el codigo en dos, esta primera parte consta de un diccionario que almacenando las clases cuyo track_id sean unicos, esto es util para almacenar el movimiento de los objetos y contabilizarlos:

In [ ]:
input_path = video_path

model = YOLO("yolov8n.pt")

# Yolo 8 permite procesar un video de forma directa
tracked_objects = model.track(source=input_path, persist=True, imgsz=1920, conf=0.6)

unique_class_counts = defaultdict(set)

for frame in tracked_objects:
    if hasattr(frame, 'boxes') and frame.boxes is not None:
        boxes = frame.boxes.xyxy.cpu() # Por defecto serian tensores
        clss = frame.boxes.cls.cpu().tolist() # Tomo las clases
        track_ids = frame.boxes.id.int().cpu().tolist() # Tomo los track_ids

        # El proposito es mantener un registro de los identificadores de seguimiento únicos para cada clase de objeto
        for cls, track_id in zip(clss, track_ids):
            unique_class_counts[int(cls)].add(track_id)

class_counts = {cls: len(track_ids) for cls, track_ids in unique_class_counts.items()}

Veamos los resultados obtenidos:

In [ ]:
class_names = {
    0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane',
    5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light',
    10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench',
    14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow',
    20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack',
    25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee',
    30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite',
    34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard',
    38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork',
    43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple',
    48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog',
    53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch',
    58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet',
    62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard',
    67: 'cell phone', 68: 'microwave', 69: 'oven', 70: 'toaster', 71: 'sink',
    72: 'refrigerator', 73: 'book', 74: 'clock', 75: 'vase', 76: 'scissors',
    77: 'teddy bear', 78: 'hair drier', 79: 'toothbrush'
}

# Ordenar el diccionario class_counts por el conteo de objetos de mayor a menor
sorted_class_counts = dict(sorted(class_counts.items(), key=lambda item: item[1], reverse=True))

# Imprimir los resultados ordenados
print('Total de objetos contabilizados:\n')
for cls, count in sorted_class_counts.items():
    class_name = class_names.get(cls, 'Unknown')
    print(f'Class {cls}: {class_name} > {count} objetos unicos')

El modelo detecto 109 personas como la clase con mas elementos unicos. Para la clase con menos objetos unicos tenemos a skate, avion, camion o bus con 1 acierto.

In [ ]:
# Obtener la clase con el mayor y menor número de objetos únicos
max_class = int(max(class_counts, key=class_counts.get))
min_class = int(min(class_counts, key=class_counts.get))

# Obtener nombre real de la clase
max_class_name = class_names.get(max_class, 'Desconocido')
min_class_name = class_names.get(min_class, 'Desconocido')

print(f'La clase única más frecuente es: {max_class_name} ({max_class}) con {class_counts[max_class]} objeto/s')
print(f'La clase única menos frecuente es: {min_class_name} ({min_class}) con {class_counts[min_class]} objeto/s')

Antes de pasar a la proxima etapa es valido aclarar que si bien la clase unica con mayores conteos fue persona, vamos a imponer la clase motocicleta como la menos frecuente para que se pueda apreciar en el video y no sea un objeto que aparece durante solo un frame, aunque la funcionalidad esta realizada, claro.

In [ ]:
input_path = video_path
output_path = "video-detection-4.mp4"

model = YOLO("yolov8n.pt")

if os.path.exists(output_path):
    os.remove(output_path)

cap = cv2.VideoCapture(input_path)
fps = cap.get(cv2.CAP_PROP_FPS)
out = imageio.get_writer(output_path, fps=fps, codec='libx264')
total_fr = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) # Para ver el progreso de avance
fr = 0

max_class_input = max_class # La resultante del código anterior
min_class_input = 3 # Forzada para notar cambios visibles

# Colores para las clases específicas
color_map = {
    max_class_input: (0, 0, 255),   # Azul para la clase de máxima frecuencia
    min_class_input: (255, 0, 0)    # Rojo para la clase de mínima frecuencia
}

while cap.isOpened():
    status, frame = cap.read()

    if status:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = model.track(frame, persist=True, imgsz=1920, classes=[max_class_input, min_class_input]) # Persistiendo tracks sobre frames
        frame_results = results[0]

        for box in frame_results.boxes:     # Persistimos sobre los resultados
            cls = int(box.cls.item())       # Obtenemos el índice de clase
            conf = box.conf.item() * 100    # Confianza en porcentaje
            obj_id = int(box.id.item())     # Identificador del objeto

            # Convertir tensor a lista y luego mapear a enteros
            x1, y1, x2, y2 = map(int, box.xyxy.cpu().tolist()[0])
            color = color_map.get(cls, (255, 255, 255))  # Color blanco por defecto

            # Obtenemos el nombre de la clase a partir del índice usando model.names
            yolo_label = model.names[cls]
            label_name = labels.get(yolo_label, 'Desconocido')  # Nombre custom o 'Desconocido'

            # Incluimos clase, ID y confianza
            text = f'{label_name} {obj_id} - {conf:.1f}%'
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)      # Dibujamos el rectangulo
            cv2.putText(frame, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2) # Manejamos las etiquetas
        fr += 1
        print(f'Frame {fr} / {total_fr}')
        out.append_data(frame)
    else:
        break

cap.release()
out.close()

In [ ]:
Video(output_path, width=1000, embed=True)

Conclusiones: La verdad que es muy divertido trabajar con video, el hecho de poder manipular la deteccion nos abre puertas para poder emplear esto, que no es nada dificil, en muchisimos ambitos del dia a dia. La ventaja de trabajar en entorno local fue gigante, dandome mas tiempo libre para mejorar el codigo y aumentar la potencia del modelo. Con respecto a Yolo, me parece una herramienta fundamental para el desarrollo de deteccion en imagenes, siendo su abanico de opciones inmenso y en constante desarrollo.
